In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torchvision
import time
from torchvision import transforms,models,datasets
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
from PIL import Image
import seaborn as sns
import numpy as np 
import pandas as pd 
import json

In [23]:
#the url of the data
url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
data = pd.read_csv(url)
#quick check on the data
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [24]:
data.loc[data['class'] == 0]['tweet'][24776]

"you're all niggers"

"you're all niggers"

In [25]:
print(data.groupby('class').size())


class
0     1430
1    19190
2     4163
dtype: int64
class
0     1430
1    19190
2     4163
dtype: int64


In [26]:
#not very clean code tho xd
no_offence=data.loc[data['class'] == 2]['tweet'].copy().reset_index(drop=True)
offence=data.loc[data['class'] == 0]['tweet'].copy().reset_index(drop=True)
X = offence[:1400].append(no_offence[:1400], ignore_index=True)
no_offenceY=data.loc[data['class'] == 2]['class'].copy().reset_index(drop=True)
offenceY=data.loc[data['class'] == 0]['class'].copy().reset_index(drop=True)
Y = offenceY[:1400].append(no_offenceY[:1400], ignore_index=True)
X.head(10)

0    "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
1    "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
2    "@DevilGrimz: @VigxRArts you're fucking gay, b...
3    "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...
4    "@NoChillPaz: "At least I'm not a nigger" http...
5    "@NotoriousBM95: @_WhitePonyJr_ Ariza is a sna...
6    "@RTNBA: Drakes new shoes that will be release...
7    "@TheoMaxximus: #GerrysHalloweenParty http://t...
8    "@ashlingwilde: @ItsNotAdam is bored supposed ...
9    "@bigbootybishopp: @white_thunduh lassen cc , ...
Name: tweet, dtype: object

0    "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
1    "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
2    "@DevilGrimz: @VigxRArts you're fucking gay, b...
3    "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...
4    "@NoChillPaz: "At least I'm not a nigger" http...
5    "@NotoriousBM95: @_WhitePonyJr_ Ariza is a sna...
6    "@RTNBA: Drakes new shoes that will be release...
7    "@TheoMaxximus: #GerrysHalloweenParty http://t...
8    "@ashlingwilde: @ItsNotAdam is bored supposed ...
9    "@bigbootybishopp: @white_thunduh lassen cc , ...
Name: tweet, dtype: object

In [27]:
from string import punctuation
for i in range(len(X)):
  X[i] = X[i].lower() # lowercase
  x= X[i].split()
  X[i]=" ".join([c for c in x if '@' not in c]) #remove all words with @, cuz it mean he/she tagged a person
  X[i] = ''.join([c for c in X[i] if c not in punctuation]) #remove punctuation

#just rechecking 
X.head(10)

0                                         queer gaywad
1    alsarabsss hes a beaner smh you can tell hes a...
2    youre fucking gay blacklisted hoe holding out ...
3    lmfaoooo i hate black people httpstcornvd2nlcd...
4     at least im not a nigger httptcorgja7cfoit lmfao
5    ariza is a snake and a coward but at least he ...
6    drakes new shoes that will be released by nike...
7    gerryshalloweenparty httptco3ycrsrnjhc hallowe...
8    is bored supposed to be cute you faggot sometimes
9    lassen cc  the coach wants me i feel it youll ...
Name: tweet, dtype: object

0                                         queer gaywad
1    alsarabsss hes a beaner smh you can tell hes a...
2    youre fucking gay blacklisted hoe holding out ...
3    lmfaoooo i hate black people httpstcornvd2nlcd...
4     at least im not a nigger httptcorgja7cfoit lmfao
5    ariza is a snake and a coward but at least he ...
6    drakes new shoes that will be released by nike...
7    gerryshalloweenparty httptco3ycrsrnjhc hallowe...
8    is bored supposed to be cute you faggot sometimes
9    lassen cc  the coach wants me i feel it youll ...
Name: tweet, dtype: object

In [28]:
from collections import Counter
words = ' '.join(X).split() #get all words in X
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
#creating our vocab (dict)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
print(vocab_to_int)
json.dump( vocab_to_int, open( "vocab_to_int.json", 'w' ) ) #saving the dic in a file


{'the': 1, 'a': 2, 'i': 3, 'you': 4, 'to': 5, 'and': 6, 'is': 7, 'in': 8, 'rt': 9, 'of': 10, 'that': 11, 'trash': 12, 'like': 13, 'my': 14, 'on': 15, 'for': 16, 'be': 17, 'are': 18, 'it': 19, 'your': 20, 'me': 21, 'bitch': 22, 'this': 23, 'faggot': 24, 'all': 25, 'im': 26, 'just': 27, 'with': 28, 'they': 29, 'dont': 30, 'u': 31, 'white': 32, 'so': 33, 'not': 34, 'have': 35, 'nigga': 36, 'ass': 37, 'but': 38, 'up': 39, 'he': 40, 'was': 41, 'at': 42, 'fuck': 43, 'its': 44, 'get': 45, 'if': 46, 'we': 47, 'nigger': 48, 'no': 49, 'lol': 50, 'what': 51, 'do': 52, 'about': 53, 'fucking': 54, 'can': 55, 'out': 56, 'yellow': 57, 'as': 58, 'amp': 59, 'his': 60, 'got': 61, 'youre': 62, 'when': 63, 'them': 64, 'bird': 65, 'or': 66, 'one': 67, 'from': 68, 'people': 69, 'some': 70, 'yankees': 71, 'go': 72, 'these': 73, 'niggas': 74, 'fag': 75, 'who': 76, 'hate': 77, 'how': 78, 'know': 79, 'thats': 80, 'will': 81, 'only': 82, 'would': 83, 'shit': 84, 'love': 85, 'man': 86, 'an': 87, 'black': 88, 'thi

In [0]:

## use the dict to tokenize each speech in speech_split
## store the tokenized speech in speech_ints
speech_ints = []
for x in X:
  speech_ints.append([vocab_to_int[word] for word in x.split()])

In [30]:
speech_lens = Counter([len(x) for x in speech_ints])
print("Zero-length speech: {}".format(speech_lens[0]))
print("Maximum review length: {}".format(max(speech_lens)))

Zero-length speech: 0
Maximum review length: 32
Zero-length speech: 0
Maximum review length: 32


In [0]:
def pad_features(speech_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    # getting the correct rows x cols shape
    features = np.zeros((len(speech_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(speech_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [0]:
seq_length = 10
features = pad_features(speech_ints, seq_length=seq_length)
assert len(features)==len(speech_ints), "Your features should have as many rows as speech."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

In [0]:
Y = Y.replace(2, 1)
# change the class from 2 to one ( for making the classification binary )
Y = np.asarray(Y)


In [34]:
split_frac = 0.8
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=42)
print(X_train.shape)

(2240, 10)
(2240, 10)


In [35]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 16


X_train = torch.stack([torch.Tensor(i) for i in X_train]) # transform to torch tensors
y_train = torch.stack([torch.Tensor([i]) for i in y_train])
X_test = torch.stack([torch.Tensor(i) for i in X_test]) # transform to torch tensors
y_test = torch.stack([torch.Tensor([i]) for i in y_test])
print(y_train)
my_dataset = TensorDataset(X_train,y_train) # create your datset
my_dataset_val = TensorDataset(X_test,y_test) # create your datset
train_loader = DataLoader(my_dataset,shuffle=True, batch_size=batch_size,drop_last=True)
val_loader = DataLoader(my_dataset_val,shuffle=True, batch_size=batch_size,drop_last=True)

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])


In [36]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.
Training on GPU.


In [0]:
import torch.nn as nn
#defining our model and archi
class SentimentRNN(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [38]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 800
hidden_dim = 300
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net = SentimentRNN(7366, 1, 800, 300, 2)

print(net)

SentimentRNN(
  (embedding): Embedding(7366, 800)
  (lstm): LSTM(800, 300, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (sig): Sigmoid()
)
SentimentRNN(
  (embedding): Embedding(7366, 800)
  (lstm): LSTM(800, 300, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
# loss and optimization functions
lr=0.0003
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters())

In [40]:
# training params

epochs = 50 
counter = 0
print_every = 500
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()
print(1)

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()
        # get the output from the model
        output, h = net(inputs, h)
        # calculate the loss and perform backprop
        loss = criterion(output.unsqueeze(dim=0), labels.reshape((1,-1)).float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        
        print("Epoch: {}/{}...".format(e+1, epochs),
              "Step: {}...".format(counter),
              "Loss: {:.6f}...".format(loss.item()))

1
Epoch: 1/50... Step: 1... Loss: 0.707122...
Epoch: 1/50... Step: 2... Loss: 0.677637...
Epoch: 1/50... Step: 3... Loss: 0.709991...
Epoch: 1/50... Step: 4... Loss: 0.678689...
Epoch: 1/50... Step: 5... Loss: 0.687534...
Epoch: 1/50... Step: 6... Loss: 0.683402...
Epoch: 1/50... Step: 7... Loss: 0.698294...
Epoch: 1/50... Step: 8... Loss: 0.681694...
Epoch: 1/50... Step: 9... Loss: 0.675058...
Epoch: 1/50... Step: 10... Loss: 0.662817...
Epoch: 1/50... Step: 11... Loss: 0.669955...
Epoch: 1/50... Step: 12... Loss: 0.665061...
Epoch: 1/50... Step: 13... Loss: 0.715382...
Epoch: 1/50... Step: 14... Loss: 0.677241...
Epoch: 1/50... Step: 15... Loss: 0.667469...
Epoch: 1/50... Step: 16... Loss: 0.626935...
Epoch: 1/50... Step: 17... Loss: 0.648045...
Epoch: 1/50... Step: 18... Loss: 0.666251...
Epoch: 1/50... Step: 19... Loss: 0.668729...
Epoch: 1/50... Step: 20... Loss: 0.647113...
Epoch: 1/50... Step: 21... Loss: 0.677948...
Epoch: 1/50... Step: 22... Loss: 0.551067...
Epoch: 1/50... St

In [43]:
# Get test data loss and accuracy
test_losses = [] # track loss
num_correct = 0
# init hidden state
h = net.init_hidden(batch_size)
net.eval()
for inputs, labels in val_loader:
    counter += 1
    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])
    # zero accumulated gradients
    # get the output from the model
    output, h = net(inputs, h)
    # calculate the loss and perform backprop
    loss = criterion(output.unsqueeze(dim=0), labels.reshape((1,-1)).float())
    test_losses.append(loss.item())
    pred = torch.round(output.squeeze())  
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(val_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))



Test loss: 1.718
Test accuracy: 0.805


In [0]:
torch.save(net.state_dict(), './model.json')
